In [1]:
# install the open cv library
!pip install opencv-python

In [15]:
# importing the packages
import pandas as pd
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# there are approx 5 features present overall across 75000 images
features_dataframe = pd.read_csv("final_dataset_containing_5_extracted_features.csv")

In [4]:
features_dataframe.shape

(72986, 12)

In [5]:
# viewing the features_dataframe
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
0,00000.png,125,101.0,73.0,NaN,NaN,89,130.0,87.0,NaN,NaN,No
1,000004.png,127,75.0,NaN,NaN,NaN,90,87.0,NaN,NaN,NaN,Yes
2,00001.png,126,75.0,76.0,NaN,NaN,91,90.0,139.0,NaN,NaN,No
3,000014.png,130,77.0,NaN,NaN,NaN,89,91.0,NaN,NaN,NaN,Yes
4,000016.png,127,173.0,74.0,NaN,NaN,90,61.0,91.0,NaN,NaN,Yes


In [6]:
# performing shuffling on features_dataframe
features_dataframe = features_dataframe.sample(frac=1)
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
61715,42127.png,127,76.0,NaN,NaN,NaN,89,90.0,NaN,NaN,NaN,No
72430,53418.png,75,125.0,100.0,81.0,NaN,90,89.0,137.0,67.0,NaN,No
5174,01577.png,77,126.0,87.0,NaN,NaN,90,90.0,139.0,NaN,NaN,No
64796,45365.png,126,86.0,74.0,NaN,NaN,86,146.0,84.0,NaN,NaN,No
15461,04646.png,120,NaN,NaN,NaN,NaN,91,NaN,NaN,NaN,NaN,No


In [7]:
features_dataframe = features_dataframe.fillna(-1)

In [8]:
# normalizing the data
is_gan = features_dataframe["is_gan"] 
features_dataframe.drop(columns=["file_name", "is_gan"], inplace=True)

In [9]:
standard_scalar = StandardScaler()
features_dataframe = standard_scalar.fit_transform(features_dataframe)
features_dataframe = pd.DataFrame(features_dataframe)
features_dataframe

,0,1,2,3,4,5,6,7,8,9
0,0.792355,-0.274241,-0.923653,-0.432465,-0.189726,-0.249139,0.156418,-0.899031,-0.420531,-0.183362
1,-1.317755,1.011883,1.081980,1.752640,-0.189726,-0.188229,0.127754,1.360817,1.104224,-0.183362
2,-1.236597,1.038130,0.823829,-0.432465,-0.189726,-0.188229,0.156418,1.393569,-0.420531,-0.183362
3,0.751776,-0.011767,0.565679,-0.432465,-0.189726,-0.431868,1.761622,0.492905,-0.420531,-0.183362
4,0.508302,-2.295293,-0.923653,-0.432465,-0.189726,-0.127320,-2.452038,-0.899031,-0.420531,-0.183362
...,...,...,...,...,...,...,...,...,...,...
72981,-1.155439,0.276955,0.903260,2.871841,5.126532,-0.005500,0.901691,1.573702,1.507836,6.467400
72982,-1.236597,-2.295293,-0.923653,-0.432465,-0.189726,-0.249139,-2.452038,-0.899031,-0.420531,-0.183362
72983,-1.236597,1.038130,1.340131,-0.432465,-0.189726,-0.188229,0.127754,1.393569,-0.420531,-0.183362
72984,0.751776,-0.169251,0.605394,2.765250,-0.189726,-0.249139,1.560971,0.591159,2.763516,-0.183362


In [10]:
features_dataframe["is_gan"] = is_gan
features_dataframe["is_gan"] = features_dataframe["is_gan"].map(dict(Yes=1, No=0))
features_dataframe

,0,1,2,3,4,5,6,7,8,9,is_gan
0,0.792355,-0.274241,-0.923653,-0.432465,-0.189726,-0.249139,0.156418,-0.899031,-0.420531,-0.183362,0
1,-1.317755,1.011883,1.081980,1.752640,-0.189726,-0.188229,0.127754,1.360817,1.104224,-0.183362,1
2,-1.236597,1.038130,0.823829,-0.432465,-0.189726,-0.188229,0.156418,1.393569,-0.420531,-0.183362,0
3,0.751776,-0.011767,0.565679,-0.432465,-0.189726,-0.431868,1.761622,0.492905,-0.420531,-0.183362,1
4,0.508302,-2.295293,-0.923653,-0.432465,-0.189726,-0.127320,-2.452038,-0.899031,-0.420531,-0.183362,1
...,...,...,...,...,...,...,...,...,...,...,...
72981,-1.155439,0.276955,0.903260,2.871841,5.126532,-0.005500,0.901691,1.573702,1.507836,6.467400,0
72982,-1.236597,-2.295293,-0.923653,-0.432465,-0.189726,-0.249139,-2.452038,-0.899031,-0.420531,-0.183362,0
72983,-1.236597,1.038130,1.340131,-0.432465,-0.189726,-0.188229,0.127754,1.393569,-0.420531,-0.183362,0
72984,0.751776,-0.169251,0.605394,2.765250,-0.189726,-0.249139,1.560971,0.591159,2.763516,-0.183362,0


In [11]:
features_dataframe.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
is_gan    0
dtype: int64

In [12]:
x = features_dataframe
y = features_dataframe["is_gan"]
x.drop(columns=["is_gan"], inplace=True)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [14]:
f"Shape of training data = {x_train.shape[0]} and testing data = {x_test.shape[0]}"

'Shape of training data = 51090 and testing data = 21896'

## Performing experiments using the following hyperparameters to find the best model

In [19]:
# Performing Kfold cross validation
knn_pipeline = Pipeline([("classifier", KNeighborsClassifier())])
param_grid = {'classifier__weights' : ['uniform', 'distance'],
             'classifier__n_neighbors' : list(range(2, 50)),
             'classifier__algorithm' : ["auto", "ball", "kd_tree", "brute"],
             'classifier__p': [1, 2],
             'classifier__n_jobs': [-1]}

knn_pipeline

Pipeline(steps=[('classifier', KNeighborsClassifier())])

In [20]:
gridsearch_cv = GridSearchCV(knn_pipeline, param_grid=param_grid, cv=10, verbose=2)

In [21]:
gridsearch_cv.fit(x_train, y_train)

Fitting 10 folds for each of 768 candidates, totalling 7680 fits
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   2.9s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   0.5s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   0.6s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=2, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__al

[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=3, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbor

[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=4, classifier__p=2, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=

[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.5s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.6s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, classifier__p=1, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=6, c

[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=1, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=1, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=2, classifier__weights=uniform; total time=   0.5s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=7, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=auto, classifier__n_jobs=-1, classifier__n_neighbors=

...

[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=uniform; total time=   0.7s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=uniform; total time=   0.7s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=uniform; total time=   0.9s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=uniform; total time=   1.0s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=uniform; total time=   1.1s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=46, classifier__p=1, classifier__weights=distance; total time=   0.6s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_

[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=1, classifier__weights=distance; total time=   1.0s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=2, classifier__weights=uniform; total time=   0.6s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=2, classifier__weights=uniform; total time=   0.6s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=47, classifier__p=2, classifier__weights=uniform; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_

[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.2s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.2s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.3s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifier__n_neighbors=48, classifier__p=2, classifier__weights=distance; total time=   0.4s
[CV] END classifier__algorithm=brute, classifier__n_jobs=-1, classifie

C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1920 fits failed out of a total of 7680.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1920 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\srmhalgi\AppData\Local\anaconda3\lib\site-packages\sklearn\neighbors\_classification.

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__algorithm': ['auto', 'ball', 'kd_tree',
                                                   'brute'],
                         'classifier__n_jobs': [-1],
                         'classifier__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                     11, 12, 13, 14, 15, 16, 17,
                                                     18, 19, 20, 21, 22, 23, 24,
                                                     25, 26, 27, 28, 29, 30, 31, ...],
                         'classifier__p': [1, 2],
                         'classifier__weights': ['uniform', 'distance']},
             verbose=2)

In [22]:
gridsearch_cv.best_params_

{'classifier__algorithm': 'brute',
 'classifier__n_jobs': -1,
 'classifier__n_neighbors': 48,
 'classifier__p': 2,
 'classifier__weights': 'uniform'}

## Training the model on the above best parameters

In [122]:
knn_algorithm = KNeighborsClassifier(algorithm="brute", n_neighbors=3,n_jobs=-1, p=2, weights="uniform")
knn_algorithm.fit(x_train, y_train)

KNeighborsClassifier(algorithm='brute', n_jobs=-1, n_neighbors=3)

In [123]:
knn_predictions = knn_algorithm.predict(x_test)

In [124]:
cm = metrics.confusion_matrix(y_test, knn_predictions)
print(f"The confusion matrix for Logistic Regression is given below: \n {cm}")

The confusion matrix for Logistic Regression is given below: 
 [[11370  3538]
 [ 5380  1608]]


In [125]:
knn_precision_score = precision_score(y_test, knn_predictions, average='weighted', labels=np.unique(knn_predictions))
knn_precision_score

0.5618934809780318

In [126]:
knn_recall = recall_score(y_test, knn_predictions, average='weighted', labels=np.unique(knn_predictions))
knn_recall

0.5927109974424553

In [127]:
knn_f1 = f1_score(y_test, knn_predictions, average='weighted', labels=np.unique(knn_predictions))
knn_f1

0.5736456972682649

In [128]:
# storing the svm model
filename = 'knn_algorithm_final.pkl'
pickle.dump(knn_algorithm, open(filename, 'wb'))

## AdaGrad Classifier

In [129]:
from lightning.classification import AdaGradClassifier

In [131]:
adagrad = AdaGradClassifier()
adagrad.fit(x_train, y_train)

AdaGradClassifier()

In [132]:
adagrad_predictions = adagrad.predict(x_test)

In [135]:
accuracy_score(adagrad_predictions, y_test)

0.5463554987212276

In [136]:
# storing the svm model
filename = 'adagrad_final.pkl'
pickle.dump(adagrad, open(filename, 'wb'))

## SGD Classifier

In [137]:
from sklearn.linear_model import SGDClassifier

In [138]:
sgd_classifier = SGDClassifier()
sgd_classifier.fit(x_train, y_train)

SGDClassifier()

In [139]:
sgd_predictions = sgd_classifier.predict(x_test)

In [141]:
accuracy_score(adagrad_predictions, y_test)

0.5463554987212276

In [142]:
# storing the svm model
filename = 'sgd_classifier_final.pkl'
pickle.dump(sgd_classifier, open(filename, 'wb'))